## CS4243, CW4, Deep Image Classification, Q5
### 2022, Amir

In [1]:
''' 
To run this code on co lab: 

add: import os

add: 
from google.colab import drive
drive.mount('/content/gdrive')
!ls

set the directory, e.g.:
"/content/gdrive/MyDrive/ANN/pets_very_small"
"/content/gdrive/MyDrive/ANN/flst.txt"

flst.txt file should be modified too

'''

' \nTo run this code on co lab: \n\nadd: import os\n\nadd: \nfrom google.colab import drive\ndrive.mount(\'/content/gdrive\')\n!ls\n\nset the directory, e.g.:\n"/content/gdrive/MyDrive/ANN/pets_very_small"\n"/content/gdrive/MyDrive/ANN/flst.txt"\n\nflst.txt file should be modified too\n\n'

In [2]:
# we just need these packages

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [4]:
# to check if we have got GPU or not

In [5]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Please install GPU version of TF


In [6]:
# setting the train and evaluation dataset and directories
# train/evaluation directory is pets_very_small 
#

In [7]:
image_size = (256,256)
batch_size = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "pets_very_small",
    validation_split=0.2,
    subset="training",
    seed=110,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "pets_very_small",
    validation_split=0.2,
    subset="validation",
    seed=110,
    image_size=image_size,
    batch_size=batch_size,
)

Found 1042 files belonging to 2 classes.
Using 834 files for training.
Found 1042 files belonging to 2 classes.
Using 208 files for validation.


In [8]:
# showing the images 
# 
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off") 

'# showing the images \n# \nimport matplotlib.pyplot as plt\n\nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(images[i].numpy().astype("uint8"))\n        plt.title(int(labels[i]))\n        plt.axis("off") '

In [9]:
# data augmentation, using horizontal flip, and random rotation 
# rotation factor is between 0 to 0.1*2pi 
# 
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [10]:
# showing the rotated and flipped images that were added to the 
# original dataset
#
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

'# showing the rotated and flipped images that were added to the \n# original dataset\n#\nplt.figure(figsize=(10, 10))\nfor images, _ in train_ds.take(1):\n    for i in range(9):\n        augmented_images = data_augmentation(images)\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(augmented_images[0].numpy().astype("uint8"))\n        plt.axis("off")  '

In [11]:
# setting train and validation datasets via augmentation

In [12]:
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [13]:
train_ds = train_ds.prefetch(buffer_size=16)
val_ds = val_ds.prefetch(buffer_size=16)

In [14]:
# making our deep model: make_model function
# basically, it is a deep convolutional netwrok. Structure is very similar with VGG16 and 19
#

In [15]:
def make_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual
    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    activation = "sigmoid"
    units = 1
   
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(25, activation='relu')(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [16]:
model = make_model(input_shape=image_size + (3,) )


In [17]:
# compiling and training our model

In [18]:
epochs = 100

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


In [19]:
model.fit(
    train_ds, epochs=epochs, validation_data=val_ds,
)

Epoch 1/5
53/53 [==============================] - 45s 817ms/step - loss: 0.7281 - accuracy: 0.5504 - val_loss: 0.6960 - val_accuracy: 0.5048
Epoch 2/5
53/53 [==============================] - 43s 801ms/step - loss: 0.7072 - accuracy: 0.5480 - val_loss: 0.7021 - val_accuracy: 0.5048
Epoch 3/5
53/53 [==============================] - 42s 800ms/step - loss: 0.6645 - accuracy: 0.5971 - val_loss: 0.7415 - val_accuracy: 0.5048
Epoch 4/5
53/53 [==============================] - 43s 809ms/step - loss: 0.6459 - accuracy: 0.6367 - val_loss: 0.8875 - val_accuracy: 0.5048
Epoch 5/5
53/53 [==============================] - 44s 831ms/step - loss: 0.6397 - accuracy: 0.6367 - val_loss: 1.0200 - val_accuracy: 0.5048


In [20]:
# above: the last validation accuracy above, is a good metric 
# to show your classifier performance 
# below: testing the classifier with some images  

In [21]:
flst = np.loadtxt('flst.txt', dtype=np.character) 
ddmm = len(flst)
tags = np.zeros( (1,ddmm) )
tags[:,27:ddmm]= 1
tags = np.int8( tags.T )

In [22]:
predct = []
for i in flst:
    img = keras.preprocessing.image.load_img( i , target_size=image_size )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = model.predict(img_array, verbose=0)
    score = predictions[0]
    print( i , 
        " is %.2f percent cat and %.2f percent dog."
        % (100 * (1 - score), 100 * score)
    )
    predct.append( np.round(score) )


b'pets_tiny_test/Cat/0.jpg'  is 84.98 percent cat and 15.02 percent dog.
b'pets_tiny_test/Cat/1.jpg'  is 85.13 percent cat and 14.87 percent dog.
b'pets_tiny_test/Cat/10.jpg'  is 85.03 percent cat and 14.97 percent dog.
b'pets_tiny_test/Cat/13.jpg'  is 84.81 percent cat and 15.19 percent dog.
b'pets_tiny_test/Cat/14.jpg'  is 85.29 percent cat and 14.71 percent dog.
b'pets_tiny_test/Cat/15.jpg'  is 85.09 percent cat and 14.91 percent dog.
b'pets_tiny_test/Cat/16.jpg'  is 84.91 percent cat and 15.09 percent dog.
b'pets_tiny_test/Cat/17.jpg'  is 85.06 percent cat and 14.94 percent dog.
b'pets_tiny_test/Cat/18.jpg'  is 84.98 percent cat and 15.02 percent dog.
b'pets_tiny_test/Cat/2.jpg'  is 84.83 percent cat and 15.17 percent dog.
b'pets_tiny_test/Cat/20.jpg'  is 84.93 percent cat and 15.07 percent dog.
b'pets_tiny_test/Cat/21.jpg'  is 84.81 percent cat and 15.19 percent dog.
b'pets_tiny_test/Cat/22.jpg'  is 85.39 percent cat and 14.61 percent dog.
b'pets_tiny_test/Cat/24.jpg'  is 84.99 pe

In [23]:
predct = np.int8( np.array(predct) )
sscc = np.sum(abs(tags-predct))
print('Number of correct classification =' , ddmm-sscc , ' out of ', ddmm , ' means ', round((ddmm-sscc)/ddmm,3) )

Number of correct classification = 27  out of  63  means  0.429


In [24]:
#model.save('resul_saved1')